# Lab | Web Scraping

Welcome to the "Books to Scrape" Web Scraping Adventure Lab!

**Objective**

In this lab, we will embark on a mission to unearth valuable insights from the data available on Books to Scrape, an online platform showcasing a wide variety of books. As data analyst, you have been tasked with scraping a specific subset of book data from Books to Scrape to assist publishing companies in understanding the landscape of highly-rated books across different genres. Your insights will help shape future book marketing strategies and publishing decisions.

**Background**

In a world where data has become the new currency, businesses are leveraging big data to make informed decisions that drive success and profitability. The publishing industry, much like others, utilizes data analytics to understand market trends, reader preferences, and the performance of books based on factors such as genre, author, and ratings. Books to Scrape serves as a rich source of such data, offering detailed information about a diverse range of books, making it an ideal platform for extracting insights to aid in informed decision-making within the literary world.

**Task**

Your task is to create a Python script using BeautifulSoup and pandas to scrape Books to Scrape book data, focusing on book ratings and genres. The script should be able to filter books with ratings above a certain threshold and in specific genres. Additionally, the script should structure the scraped data in a tabular format using pandas for further analysis.

**Expected Outcome**

A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`. The function should scrape book data from the "Books to Scrape" website and return a `pandas` DataFrame with the following columns:

**Expected Outcome**

- A function named `scrape_books` that takes two parameters: `min_rating` and `max_price`.
- The function should return a DataFrame with the following columns:
  - **UPC**: The Universal Product Code (UPC) of the book.
  - **Title**: The title of the book.
  - **Price (£)**: The price of the book in pounds.
  - **Rating**: The rating of the book (1-5 stars).
  - **Genre**: The genre of the book.
  - **Availability**: Whether the book is in stock or not.
  - **Description**: A brief description or product description of the book (if available).
  
You will execute this script to scrape data for books with a minimum rating of `4.0 and above` and a maximum price of `£20`. 

Remember to experiment with different ratings and prices to ensure your code is versatile and can handle various searches effectively!

**Resources**

- [Beautiful Soup Documentation](https://www.crummy.com/software/BeautifulSoup/bs4/doc/)
- [Pandas Documentation](https://pandas.pydata.org/pandas-docs/stable/index.html)
- [Books to Scrape](https://books.toscrape.com/)


**Hint**

Your first mission is to familiarize yourself with the **Books to Scrape** website. Navigate to [Books to Scrape](http://books.toscrape.com/) and explore the available books to understand their layout and structure. 

Next, think about how you can set parameters for your data extraction:

- **Minimum Rating**: Focus on books with a rating of 4.0 and above.
- **Maximum Price**: Filter for books priced up to £20.

After reviewing the site, you can construct a plan for scraping relevant data. Pay attention to the details displayed for each book, including the title, price, rating, and availability. This will help you identify the correct HTML elements to target with your scraping script.

Make sure to build your scraping URL and logic based on the patterns you observe in the HTML structure of the book listings!


---

**Best of luck! Immerse yourself in the world of books, and may the data be with you!**

**Important Note**:

In the fast-changing online world, websites often update and change their structures. When you try this lab, the **Books to Scrape** website might differ from what you expect.

If you encounter issues due to these changes, like new rules or obstacles preventing data extraction, don’t worry! Get creative.

You can choose another website that interests you and is suitable for scraping data. Options like Wikipedia, The New York Times, or even library databases are great alternatives. The main goal remains the same: extract useful data and enhance your web scraping skills while exploring a source of information you enjoy. This is your opportunity to practice and adapt to different web environments!

In [216]:
# Let's set up the libraries and import all nesessary things
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [217]:
url = "https://books.toscrape.com/catalogue/page-1.html"
response = requests.get(url)

In [221]:
# Getting the list of all individual book URL's. The function loops trough all 50 pages
# From each of the page, function extracts the URL for 20 books.
# Retunr the complet list of 1000 URL's
base_url = "https://books.toscrape.com/catalogue/"
all_book_urls = []

# Loop through all 50 pages
for page_num in range(1, 51):
    url = f"{base_url}page-{page_num}.html"
    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    grid = soup.find("ol", attrs = {"class":"row"})
    books = grid.find_all("li", attrs = {"class":"col-xs-6 col-sm-4 col-md-3 col-lg-3"})

    for book in books: # Loop trough all books per page
        book_link = book.find("h3").find("a")["href"]
        full_link = base_url + book_link
        all_book_urls.append(full_link)

In [218]:
# Let's define the functions to get the required data
# All data is retrieved using the individual URL's
def get_upc(book):
    """
    Function to get the book UPC
    """
    soup = BeautifulSoup(response.content)
    upc = soup.find("table", attrs = {"class":"table table-striped"}).find("tr").find("td").text.strip()
    return upc

def get_title(book):
    """
    Function to get the book title
    """
    soup = BeautifulSoup(response.content)
    main = soup.find("div", attrs = {"class":"col-sm-6 product_main"})
    title = main.find("h1").get_text()
    return title

def get_price(book):
    """
    Function to get the book price
    """
    soup = BeautifulSoup(response.content)
    main = soup.find("div", attrs = {"class":"col-sm-6 product_main"})
    price = float(main.find("p", attrs = {"class":"price_color"}).get_text().replace("£",""))
    return price

def get_rating(book):
    """
    Function to get the book rating
    """
    soup = BeautifulSoup(response.content)
    main = soup.find("div", attrs = {"class":"col-sm-6 product_main"})
    rating = main.find("p", class_="star-rating").get("class")[1]
    rating_map = {
    "One": 1,
    "Two": 2,
    "Three": 3,
    "Four": 4,
    "Five": 5
    }
    rating = rating_map.get(rating)
    return rating

def get_genre(book):
    """
    Function to get the book genre
    """
    soup = BeautifulSoup(response.content)
    genre = soup.find("ul", {"class":"breadcrumb"}).find_all("li")[2].find("a").text.strip()
    return genre

def get_availability(book):
    """
    Function to get the book availability
    """
    soup = BeautifulSoup(response.content)
    main = soup.find("div", attrs = {"class":"col-sm-6 product_main"})
    availability = main.find("p", attrs = {"class":"instock availability"}).text.strip()
    return availability

def get_description(book):
    """
    Function to get the book description
    """
    soup = BeautifulSoup(response.content)
    description = soup.find("div", attrs = {"id":"product_description"}).find_next_sibling("p").text.strip()
    return description

In [ ]:
#Lets iterate our functions to all book urls, and save the retrieved data in a dataframen
min_rating = 4 
max_price = 20.0

books_dict = {}
key = 0

for book in all_book_urls:
    response = requests.get(book)

    rating = get_rating(book)
    if rating < min_rating:
        continue
    else:
        price = get_price(book)
        if price > max_price:
            continue
        else:
            upc = get_upc(book)
            title = get_title(book)
            genre = get_genre(book)
            availability = get_availability(book)
            description = get_description(book)
        
            books_dict[key] = {"upc": upc,
                               "title": title,
                               "price": price,
                               "rating": rating,
                               "genre": genre,
                               "availability": availability,
                               "description": description
                              }
            key += 1


books_df = pd.DataFrame.from_dict(books_dict, orient = "index")
books_df